# <b>Python for Data Analysis</b>
# 6.  Data Loading, Storage, and File Formats

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)
from pandas import Series, DataFrame

## 6.1 Reading and Writing Data in Text Format

In [5]:
df = pd.read_csv("examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv("examples/ex2.csv", header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
pd.read_csv("examples/ex2.csv", names = ["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
names = ["a", "b", "c", "d", "message"]

In [12]:
pd.read_csv("examples/ex2.csv", names = names, index_col = "message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [13]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col = ["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [14]:
result = pd.read_csv("examples/ex3.txt", sep = "\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
pd.read_csv("examples/ex4.csv", skiprows = [0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
result = pd.read_csv("examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
result = pd.read_csv("examples/ex5.csv", na_values = ["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na = False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [21]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [22]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na = False,
                     na_values = ["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [23]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [24]:
sentinels = {"message":   ["foo", "NA"],
             "something": ["two"]}

In [26]:
pd.read_csv("examples/ex5.csv", 
            na_values = sentinels,
            keep_default_na = False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Reading Text Files in Pieces

In [33]:
pd.options.display.max_rows = 10

In [34]:
result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [37]:
pd.read_csv("examples/ex6.csv", nrows = 5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [38]:
chunker = pd.read_csv("examples/ex6.csv", chunksize = 1000)

In [40]:
type(chunker)

pandas.io.parsers.readers.TextFileReader

In [41]:
tot = pd.Series([], dtype = 'int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value = 0)
    
tot = tot.sort_values(ascending = False)

In [42]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

In [43]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [44]:
data.to_csv("examples/out.csv")

In [45]:
import sys

In [47]:
data.to_csv(sys.stdout, sep = "|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [48]:
data.to_csv(sys.stdout, na_rep = "NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [49]:
data.to_csv(sys.stdout, index = False, header = False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [50]:
data.to_csv(sys.stdout, index = False, columns = ["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Working with Other Delimited Formats

In [51]:
import csv

In [65]:
f = open("examples/ex7.csv")
reader = csv.reader(f)

In [54]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [71]:
f.close()

In [56]:
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))

In [58]:
header, values = lines[0], lines[1:]

In [59]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [66]:
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [67]:
reader = csv.reader(f, dialect = my_dialect)

In [70]:
reader = csv.reader(f, delimiter = "|")

In [72]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect = my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

### JSON Data

In [73]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [74]:
import json

In [75]:
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [76]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [78]:
siblings = pd.DataFrame(result["siblings"], columns = ["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [79]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [80]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [81]:
data.to_json(sys.stdout, orient = "records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

### XML and HTML: Web Scraping

In [82]:
conda install lxml beautifulsoup4 html5lib

^C

Note: you may need to restart the kernel to use updated packages.


In [84]:
pip install lxml

^C
Note: you may need to restart the kernel to use updated packages.


In [88]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [89]:
len(tables)

1

In [90]:
failures = tables[0]

In [91]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [94]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with <code>lxml.objectify</code>

In [95]:
from lxml import objectify

In [98]:
path = "datasets/mta_perf/Performance_MNR.xml"

In [99]:
with open(path) as f:
    parsed = objectify.parse(f)

In [100]:
root = parsed.getroot()

In [101]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [102]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [103]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


## 6.2 Binary Data Formats

In [104]:
frame = pd.read_csv("examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [105]:
frame.to_pickle("examples/frame_pickle")

In [106]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [108]:
conda install pyarrow

^C

Note: you may need to restart the kernel to use updated packages.


In [111]:
# fec = pd.read_parquet('datasets/fec/fec.parquet')

### Reading Microsoft Excel Files

In [112]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [113]:
xlsx.sheet_names

['Sheet1']

In [114]:
xlsx.parse(sheet_name = "Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [115]:
xlsx.parse(sheet_name = "Sheet1", index_col = 0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [116]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name = "Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [117]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

In [118]:
frame.to_excel("examples/ex2.xlsx")


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2022.10=py39_0
  - defaults/win-64::anaconda-client==1.11.0=py39haa95532_0
  - defaults/win-64::anaconda-navigator==2.4.0=py39haa95532_0
  - defaults/win-64::anaconda-project==0.11.1=py39haa95532_0
  - defaults/noarch::babel==2.9.1=pyhd3eb1b0_0
  - defaults/win-64::bkcharts==0.2=py39haa95532_1
  - defaults/win-64::colorcet==3.0.0=py39haa95532_0
  - defaults/win-64::conda==23.1.0=py39haa95532_0
  - defaults/win-64::conda-build==3.22.0=py39haa95532_0
  - defaults/win-64::conda-repo-cli==1.0.20=py39haa95532_0
  - defaults/noarch::conda-token==0.4.0=pyhd3eb1b0_0
  - defaults/noarch::cookiecutter==1.7.3=pyhd3eb1b0_0
  - defaults/win-64::dask==2022.7.0=py39haa95532_0
  - defaults/win-64::datashader==0.14.1=py39haa95532_0
  - defaults/win-64::holoviews==1.15.0=py39haa95532_0
  - defaults/win-64::hvplot==0.8.0=py39haa95532_0
  - default

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - pyarrow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    arrow-cpp-8.0.0            |   py39hbd6f097_0         6.2 MB
    aws-c-common-0.4.57        |       ha925a31_1         147 KB
    aws-c-event-stream-0.1.6   |       hd77b12b_5          26 KB
    aws-checksums-0.1.9        |       ha925a31_0          50 KB
    aws-sdk-cpp-1.8.185        |       hd77b12b_0         2.5 MB
    boost-cpp-1.73.0           |      h2bbff1b_12          16 KB
    libboost-1.73.0            |      h6c2663c_12        20.1 MB
    libprotobuf-3.20.1      

### Using HDF5 Format

In [119]:
conda install pytables

^C

Note: you may need to restart the kernel to use updated packages.


In [120]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})

In [121]:
store = pd.HDFStore("examples/mydata.h5")

In [122]:
store["obj1"] = frame

In [123]:
store["obj1_col"] = frame["a"]

In [124]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [125]:
store["obj1"]

,a
0,-0.023412
1,0.348620
2,0.123364
3,0.949333
4,-0.437831
...,...
95,0.669378
96,0.688773
97,-1.484142
98,1.705584


In [126]:
store.put("obj2", frame, format = "table")

In [128]:
store.select("obj2", where = ["index >= 10 and index <= 15"])

,a
10,-0.286070
11,0.113897
12,-0.409026
13,-0.417292
14,-0.505407
15,1.218528


In [129]:
store.close()

In [132]:
frame.to_hdf("examples/mydata.h5", "obj3", format = "table")

In [133]:
pd.read_hdf("examples/mydata.h5", "obj3", where = ["index < 5"])

,a
0,-0.023412
1,0.348620
2,0.123364
3,0.949333
4,-0.437831


In [ ]:
# Delete the created HDF5 file:
# import os
# os.remove("examples/mydata.h5")

## 6.3 Interacting with Web APIs

In [2]:
import requests

In [3]:
url = "https://api.github.com/repos/pandas-dev/pandas/issues"

In [4]:
resp = requests.get(url)

In [5]:
resp.raise_for_status()

In [6]:
resp

<Response [200]>

In [7]:
data = resp.json()

In [8]:
data[0]["title"]

'Backport PR #57034 on branch 2.2.x (REGR: perf regression in Series.combine_first)'

In [15]:
data[0]

{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57072',
 'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
 'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57072/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57072/comments',
 'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57072/events',
 'html_url': 'https://github.com/pandas-dev/pandas/pull/57072',
 'id': 2100745152,
 'node_id': 'PR_kwDOAA0YD85lFYz0',
 'number': 57072,
 'title': 'Backport PR #57034 on branch 2.2.x (REGR: perf regression in Series.combine_first)',
 'user': {'login': 'meeseeksmachine',
  'id': 39504233,
  'node_id': 'MDQ6VXNlcjM5NTA0MjMz',
  'avatar_url': 'https://avatars.githubusercontent.com/u/39504233?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/meeseeksmachine',
  'html_url': 'https://github.com/meeseeksmachine',
  'followers_url': 'https://api.github.com/users/meeseeksmachine/fo

In [16]:
issues = pd.DataFrame(data,
                      columns = ["number", "title", "labels", "state"])
issues

,number,title,labels,state
0,57072,Backport PR #57034 on branch 2.2.x (REGR: perf regression in Series.combine_...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1MzEx', 'url': 'https://api.gith...",open
1,57071,BUG: read_csv: on_bad_lines= callable does not call handler function,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
2,57070,BUG: Fix reading of multi_index dataframe with NaN,[],open
3,57069,BUG: DataFrameGroupBy.transform with engine='numba' reorder output by index,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
4,57068,BUG: Regression from v2.1.4 to v2.2.0 when querying pd.NaT/timestamp compari...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
...,...,...,...,...
25,57040,BUG: unexpected results for `idxmax` groupby aggregation on uint column,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
26,57039,"BUG: pandas 2.2 read_csv(engine=""c"") leaks memory when code uses np.nan","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
27,57038,BUILD: Can't found module wrapt nor detect right meson version,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNTA=', 'url': 'https://api.githu...",open
28,57037,BUG: Wrong column types when query has no rows in read_sql,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open


## 6.4 Interacting with Databases

In [17]:
import sqlite3

In [18]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL,         d INTEGER
);"""

In [19]:
con = sqlite3.connect("mydata.sqlite")

In [20]:
con.execute(query)

In [21]:
con.commit()

In [22]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]

In [23]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [24]:
con.executemany(stmt, data)

In [25]:
con.commit()

In [26]:
cursor = con.execute("SELECT * FROM test")

In [27]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [28]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [29]:
pd.DataFrame(rows, columns = [x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [30]:
import sqlalchemy as sqla

In [31]:
db = sqla.create_engine("sqlite:///mydata.sqlite")

In [32]:
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
